In [4]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

In [5]:
#df=pd.read_csv(r"C:\Users\LALITHA\...My Data\Projects\Preethi Twitter Sentiment Analysis\train.csv",encoding='ISO-8859-1')
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_c10f99a74fa048d5995e3c3b29070787 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='iMkGzfAxt28C487jaht_VYgpFJYO96zE2vEkIzFRe30o',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_c10f99a74fa048d5995e3c3b29070787.get_object(Bucket='annregression-donotdelete-pr-k9wxg0xahqhvtd',Key='train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body,encoding='ISO-8859-1')
df.head()

#df.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [6]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
df['SentimentText'][0]
copy=df['SentimentText']
c=[]


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
for i in range(0,99988):
    review = re.sub('[^a-zA-Z]',' ',df['SentimentText'][i])
    review=review.lower()
    review=review.split()
    ps=PorterStemmer()
    review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review= ' '.join(review)
    c.append(review)
    

In [8]:
from sklearn.feature_extraction.text import CountVectorizer  
cv = CountVectorizer(max_features=1500)
x=cv.fit_transform(c).toarray()
with open("CountVectorizer",'wb') as f:
    pickle.dump(cv,f)
y=df.iloc[:,1].values

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=0)


In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [11]:
cls=Sequential()

In [12]:
cls.add(Dense(output_dim=1000,init='uniform', activation='sigmoid',input_dim=1500))


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1000, input_dim=1500, activation="sigmoid", kernel_initializer="uniform")`
  if __name__ == '__main__':


In [13]:
cls.add(Dense(output_dim=100,init='uniform', activation='sigmoid'))


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=100, activation="sigmoid", kernel_initializer="uniform")`
  if __name__ == '__main__':


In [14]:
cls.add(Dense(output_dim=1,init='uniform', activation='sigmoid'))


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="sigmoid", kernel_initializer="uniform")`
  if __name__ == '__main__':


In [15]:
cls.compile(optimizer='adam',loss='binary_crossentropy')


In [16]:
cls.fit(x_train,y_train,epochs=50,batch_size=50)


Epoch 1/50
89989/89989 [==============================] - 208s 2ms/step - loss: 0.5566
Epoch 2/50
89989/89989 [==============================] - 222s 2ms/step - loss: 0.5217
Epoch 3/50
89989/89989 [==============================] - 235s 3ms/step - loss: 0.5150
Epoch 4/50
89989/89989 [==============================] - 246s 3ms/step - loss: 0.5090
Epoch 5/50
89989/89989 [==============================] - 243s 3ms/step - loss: 0.5060
Epoch 6/50
89989/89989 [==============================] - 246s 3ms/step - loss: 0.5036 0s - loss: 0.5
Epoch 7/50
89989/89989 [==============================] - 246s 3ms/step - loss: 0.5011
Epoch 8/50
89989/89989 [==============================] - 252s 3ms/step - loss: 0.4989
Epoch 9/50
89989/89989 [==============================] - 250s 3ms/step - loss: 0.4979
Epoch 10/50
89989/89989 [==============================] - 251s 3ms/step - loss: 0.4957
Epoch 11/50
89989/89989 [==============================] - 249s 3ms/step - loss: 0.4933
Epoch 12/50
89989/89989 [=

In [17]:
cls.save('mymodel.h5')


In [18]:

from keras.models import load_model

In [20]:
m = load_model('mymodel.h5')
with open('CountVectorizer','rb') as file:
    cv=pickle.load(file)

In [21]:
x_intent="bad mood"
x_intent=cv.transform([x_intent])
y_pred=m.predict(x_intent)
y_pred=(y_pred>0.5)
print(y_pred)

[[False]]


In [22]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [23]:
wml_credentials={
  "apikey": "CYDmOKrW1LOeRbnI23uH9LT7KLRQ2kMXIj0EgNu2mLcn",
   "instance_id": "971c927c-e6eb-490c-aa7d-5e5915604451",
  "password": "56129d8e-a3e9-41e8-ba96-5f0a9f1772cc",
  "url": "https://eu-gb.ml.cloud.ibm.com",
  "username": "05c031df-7741-4b8a-911a-427339805b8c"
}

In [24]:
client = WatsonMachineLearningAPIClient( wml_credentials )

In [25]:
metadata = {
    client.repository.ModelMetaNames.NAME: "keras",
    client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [{'name':'keras', 'version': '2.1.3'}],
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.5",
    client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow"
}

In [26]:
!tar -zcvf mymodel.tgz mymodel.h5

mymodel.h5


In [27]:
model_details = client.repository.store_model( model="mymodel.tgz", meta_props=metadata )


In [28]:
model_id = model_details["metadata"]["guid"]
model_deployment_details = client.deployments.create( artifact_uid=model_id, name="deployment" )




#######################################################################################

Synchronous deployment creation for uid: 'c17923f8-a916-419d-8a00-1298f1f4e65c' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='d391d761-61ea-45bf-be42-071d3d787117'
------------------------------------------------------------------------------------------------




In [29]:
scoring_endpoint=client.deployments.get_scoring_url(model_deployment_details)


In [30]:
scoring_endpoint

'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/971c927c-e6eb-490c-aa7d-5e5915604451/deployments/d391d761-61ea-45bf-be42-071d3d787117/online'